BI Goals:

i) investigar as ligações entre a atividade do fogo, conforme medido pelo FRP (Fire Radiative Power), e as concentrações de poluentes e avaliar a zona espacial e temporal de influência da atividade dos incêndios florestais.

(ii) investigar a utilização de FRP como ferramenta para filtrar a contribuição do fumo de biomassa para os registos de poluição atmosférica em bacias atmosféricas urbanas, nomeadamente as emissões de carbono resultantes de incêndios florestais graves.

(iii) desenvolver abordagens multirriscos para caracterizar o comportamento conjunto de múltiplos perigos e riscos consequentes e avaliar o papel desempenhado por condições anteriores e simultâneas de seca e/ou calor na exacerbação de incêndios rurais e consequentes ondas de fumo

In [ ]:
# Gaussian Plume Model Implementation with Supporting Code
# ---------------------------------------------------------
# Assumes user has gridded daily FRP (MW) and wind data (ERA5 u10, v10), pollutant concentrations (CAMS), and grid/mask data.

import numpy as np
import xarray as xr
import pandas as pd
import math

# -----------------------------
# 1. Compute Wind Speed and Direction from u and v
# -----------------------------
def compute_wind_speed_direction(u10, v10):
    wind_speed = np.sqrt(u10**2 + v10**2)
    wind_dir = (np.arctan2(-u10, -v10) * 180 / np.pi) % 360
    return wind_speed, wind_dir

# -----------------------------
# 2. Estimate Atmospheric Stability Class
# -----------------------------
def classify_stability(wind_speed, cloud_cover, ssrd, hour):
    if hour >= 6 and hour <= 18:  # daytime
        if ssrd > 700 and wind_speed < 2:
            return 'A'
        elif ssrd > 500:
            return 'B'
        elif ssrd > 250:
            return 'C'
        else:
            return 'D'
    else:  # nighttime
        if cloud_cover > 0.5:
            return 'D'
        elif wind_speed < 2:
            return 'F'
        else:
            return 'E'

# -----------------------------
# 3. Compute Dispersion Coefficients
# -----------------------------
def dispersion_coefficients(x, stability):
    if stability == 'A':
        sigma_y = 0.22 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.20 * x
    elif stability == 'B':
        sigma_y = 0.16 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.12 * x
    elif stability == 'C':
        sigma_y = 0.11 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.08 * x * (1 + 0.0002 * x)**-0.5
    elif stability == 'D':
        sigma_y = 0.08 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.06 * x * (1 + 0.0015 * x)**-0.5
    elif stability == 'E':
        sigma_y = 0.06 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.03 * x * (1 + 0.0003 * x)**-1
    elif stability == 'F':
        sigma_y = 0.04 * x * (1 + 0.0001 * x)**-0.5
        sigma_z = 0.016 * x * (1 + 0.0003 * x)**-1
    else:
        sigma_y = sigma_z = np.nan
    return sigma_y, sigma_z

# -----------------------------
# 4. Estimate Emission Rate Q from FRP (MW) and Emission Factor (EF in g/MJ)
# -----------------------------
def compute_emission_rate(FRP_MW, EF_g_per_MJ):
    # Convert FRP from MW to MJ/s
    FRP_MJ_s = FRP_MW * 1000
    Q_grams_per_s = FRP_MJ_s * EF_g_per_MJ
    Q_kg_per_s = Q_grams_per_s / 1000
    return Q_kg_per_s

# -----------------------------
# 5. Gaussian Plume Model (ground-level, steady-state)
# -----------------------------
def gaussian_plume_ground(Q, u, x, y, H, sigma_y, sigma_z):
    factor = Q / (2 * np.pi * u * sigma_y * sigma_z)
    exp_y = np.exp(- (y**2) / (2 * sigma_y**2))
    exp_z = np.exp(- (H**2) / (2 * sigma_z**2))
    C = factor * exp_y * exp_z
    return C

# -----------------------------
# 6. Apply Regional Mask for Aggregation
# -----------------------------
def apply_region_mask(concentration, mask):
    masked = np.where(mask == 1, concentration, np.nan)
    regional_mean = np.nanmean(masked)
    return regional_mean

# -----------------------------
# 7. Main Modeling Loop (Example for One Day, One Pollutant)
# -----------------------------
def run_model_for_day(FRP_ds, u10_ds, v10_ds, ssrd_ds, cloud_ds, mask_ds, EF, H=30):
    results = []
    lats = FRP_ds.latitude.values
    lons = FRP_ds.longitude.values

    for i, lat in enumerate(lats):
        for j, lon in enumerate(lons):
            FRP = FRP_ds.FRP_sum.values[0, i, j]
            if np.isnan(FRP) or FRP == 0:
                continue

            u10 = u10_ds.u10.values[0, i, j]
            v10 = v10_ds.v10.values[0, i, j]
            ssrd = ssrd_ds.ssrd.values[0, i, j]
            cloud = cloud_ds.tcc.values[0, i, j]

            wind_speed, wind_dir = compute_wind_speed_direction(u10, v10)
            stability = classify_stability(wind_speed, cloud, ssrd, hour=12)  # Assuming noon
            Q = compute_emission_rate(FRP, EF)

            x = 10000  # meters downwind
            y = 0      # centerline
            sigma_y, sigma_z = dispersion_coefficients(x, stability)

            C = gaussian_plume_ground(Q, wind_speed, x, y, H, sigma_y, sigma_z)
            results.append({"lat": lat, "lon": lon, "C": C})

    return pd.DataFrame(results)


In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from scipy.stats import pearsonr

# Define your region of interest (Portugal example)
region_bounds = {"lat_min": 36, "lat_max": 43, "lon_min": -10, "lon_max": -6}

# Load your datasets
frp_ds = xr.open_dataset("E:/IPMA/FRP/FRP_2000_2024_grid.nc")
pollutant_ds = xr.open_dataset("E:/IPMA/CAMS/chem_singlvl/daily_pm2p5_stats.nc")  # for PM2.5 example
meteo_ds = xr.open_dataset("E:/IPMA/CAMS/meteo_multlvl/ERA5_daily_temperature.nc")  # replace with your ERA5 daily temp


# Regional average FRP time series
frp_region = frp_ds.sel(
    latitude=slice(region_bounds["lat_max"], region_bounds["lat_min"]),
    longitude=slice(region_bounds["lon_min"], region_bounds["lon_max"])
)
daily_frp = frp_region['frp'].mean(dim=['latitude', 'longitude'])

# Identify "fire days" above a threshold (e.g., 90th percentile)
threshold = np.percentile(daily_frp.values, 90)
fire_days = daily_frp.time[daily_frp > threshold].values

print(f"Number of high-FRP days: {len(fire_days)}")


def extract_lagged_pollution(ds, variable='Mean', lags=[-3, -2, -1, 0, 1, 2, 3]):
    results = {lag: [] for lag in lags}
    for date in pd.to_datetime(fire_days):
        for lag in lags:
            lagged_date = np.datetime64(date + timedelta(days=lag))
            try:
                value = ds.sel(time=lagged_date).sel(
                    latitude=slice(region_bounds["lat_max"], region_bounds["lat_min"]),
                    longitude=slice(region_bounds["lon_min"], region_bounds["lon_max"])
                )[variable].mean().item()
                results[lag].append(value)
            except:
                results[lag].append(np.nan)
    return pd.DataFrame(results)

# Extract PM2.5 around fire days
pm25_lagged = extract_lagged_pollution(pollutant_ds)

# Plot mean response
pm25_lagged.mean().plot(marker='o')
plt.axvline(0, color='red', linestyle='--', label='Fire Day')
plt.title("PM2.5 Evolution Around Fire Days (Portugal)")
plt.xlabel("Days relative to fire")
plt.ylabel("PM2.5 (µg/m³)")
plt.grid(True)
plt.legend()
plt.show()


# Align all datasets to same time period
common_time = np.intersect1d(pollutant_ds.time.values, frp_ds.time.values)
frp_avg = daily_frp.sel(time=common_time)
pm25_avg = pollutant_ds['Mean'].sel(
    time=common_time,
    latitude=slice(region_bounds["lat_max"], region_bounds["lat_min"]),
    longitude=slice(region_bounds["lon_min"], region_bounds["lon_max"])
).mean(dim=['latitude', 'longitude'])

# Compute correlation
corr, pval = pearsonr(frp_avg.values, pm25_avg.values)
print(f"Correlation between FRP and PM2.5: r = {corr:.2f}, p = {pval:.3f}")
